# TODO

1. Limpiar los genres de los animes

In [1]:
import os
import pandas as pd

In [2]:
CWD = os.getcwd()

In [3]:
PATH_INPUT_FOLDER = os.path.join(CWD, "input")

In [4]:
PATH_INPUT_FOLDER

'/Users/nicolaepopescul/Desktop/streams/20240511_content_based_recommenders/input'

In [5]:
os.listdir(PATH_INPUT_FOLDER)

['cf_anime.parquet.gzip', 'cf_rating.parquet.gzip']

In [6]:
rating = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_rating.parquet.gzip'))
anime = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_anime.parquet.gzip'))

In [7]:
rating = rating[rating["rating"] != -1]

In [8]:
rating.sample(5)

,user_id,anime_id,rating
5377732,50877,269,9
3481985,32185,13469,7
9668,123,7857,6
4401745,41544,2476,7
1905401,18512,1818,8


In [9]:
anime = anime[["anime_id", "name", "genre"]]

In [10]:
anime.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


In [11]:
genres = anime["genre"].str.split(", ", expand = True)

In [12]:
anime = pd.concat([anime, genres], axis = 1)

In [13]:
anime.head(3)

,anime_id,name,genre,0,1,2,3,4,5,6,7,8,9,10,11,12
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Drama,Romance,School,Supernatural,None,None,None,None,None,None,None,None,None
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,None,None,None,None,None,None
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",Action,Comedy,Historical,Parody,Samurai,Sci-Fi,Shounen,None,None,None,None,None,None


In [14]:
anime_genre = (
    anime[["anime_id"] + genres.columns.tolist()]
    .melt(id_vars = "anime_id")
    [["anime_id", "value"]]
    .dropna()
    .pivot_table(
        index = "anime_id",
        columns = "value",
        aggfunc = len,
        fill_value = 0
    )
)

In [15]:
anime_genre.head(3)

value,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
oc = [
    'user_id',
    'name',
    'rating',
    'Action',
    'Adventure',
    'Cars',
    'Comedy',
    'Dementia',
    'Demons',
    'Drama',
    'Ecchi',
    'Fantasy',
    'Game',
    'Harem',
    'Hentai',
    'Historical',
    'Horror',
    'Josei',
    'Kids',
    'Magic',
    'Martial Arts',
    'Mecha',
    'Military',
    'Music',
    'Mystery',
    'Parody',
    'Police',
    'Psychological',
    'Romance',
    'Samurai',
    'School',
    'Sci-Fi',
    'Seinen',
    'Shoujo',
    'Shoujo Ai',
    'Shounen',
    'Shounen Ai',
    'Slice of Life',
    'Space',
    'Sports',
    'Super Power',
    'Supernatural',
    'Thriller',
    'Vampire',
    'Yaoi',
    'Yuri'
]

In [17]:
X = pd.merge(
    left = rating,
    right = anime_genre.reset_index(),
    how = "inner",
    on = ["anime_id"]
)

In [18]:
X = X.merge(right = anime[["anime_id", "name"]], how = "inner", on = ["anime_id"])

In [ ]:
X = X[oc]

In [ ]:
X.sample(10)

In [ ]:
X.iloc[:, 3::] = X.iloc[:, 3::].multiply(X["rating"], axis = 0)

In [ ]:
X.sample(10)

In [ ]:
ponderated_score = (
    X
    .drop(columns = ["name", "rating"])
    .groupby(["user_id"])
    .sum()
    .pipe(lambda df: df.divide(df.sum(axis = 1), axis = 0))
)

In [ ]:
ponderated_score

# Parte de Inferencia

In [ ]:
random_user = pd.Series(rating["user_id"].unique()).sample().iloc[0]

In [ ]:
random_user

In [ ]:
random_animes = anime_genre.reset_index()["anime_id"].sample(10).values.tolist()

In [ ]:
random_animes

In [ ]:
random_animes_genres = anime_genre[anime_genre.index.isin(random_animes)]

In [ ]:
random_animes_genres

In [ ]:
random_user_ponderated_score = ponderated_score[ponderated_score.index.isin([random_user])]

In [ ]:
random_user_ponderated_score

In [ ]:
r = (
    random_user_ponderated_score.dot(random_animes_genres.T)
    .T
    .rename(columns = {random_user:"score"})
    .sort_values(by = "score", ascending = False)
    # .pipe(lambda df: pd.concat([df, random_animes_genres], axis = 1))
)

In [ ]:
r

In [ ]:
(
    random_user_ponderated_score
    .melt()
    .sort_values("value", ascending = False)
)

In [ ]:
i = 1
(
    random_animes_genres.loc[r.index[i]][random_animes_genres.loc[r.index[i]] != 0]
)